In [ ]:
# default_exp pod.api
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
import os
from typing import List, Iterator
import requests

In [ ]:
# export
DEFAULT_POD_ADDRESS = os.environ.get("POD_ADDRESS") or "http://localhost:3030"
POD_VERSION = "v4"

In [ ]:
# export
class PodAPI:
    def __init__(
        self,
        database_key: str,
        owner_key: str,
        url: str = DEFAULT_POD_ADDRESS,
        version: str = POD_VERSION,
        auth_json: dict = None,
    ):
        self.database_key = database_key
        self.owner_key = owner_key

        self._url = url
        self.base_url = f"{url}/{version}/{self.owner_key}"

        if auth_json is not None:
            self.auth_json = {"type": "PluginAuth", **auth_json}
        else:
            self.auth_json = {"type": "ClientAuth", "databaseKey": self.database_key}

    @property
    def version(self):
        response = requests.get(f"{self._url}/version")
        response.raise_for_status()
        return response.json()

    def post(self, endpoint, payload):
        body = {"auth": self.auth_json, "payload": payload}
        response = requests.post(f"{self.base_url}/{endpoint}", json=body)
        response.raise_for_status()
        return response.json()

    def get_item(self, uid: str) -> dict:
        return self.post("get_item", uid)

    def create_item(self, item: dict) -> str:
        return self.post("create_item", item)

    def update_item(self, item: dict) -> None:
        return self.post("update_item", item)

    def get_edges(
        self, uid: str, direction: str = "Outgoing", expand_items: bool = True
    ) -> List[dict]:
        payload = {"item": uid, "direction": direction, "expandItems": expand_items}
        return self.post("get_edges", payload)

    def create_edge(self, src_id: str, tgt_id: str, name: str) -> str:
        payload = {"_source": src_id, "_target": tgt_id, "_name": name}
        return self.post("create_edge", payload)
        
    def delete_item(self, uid) -> None:
        return self.post("delete_item", uid)

    def search(self, query: dict) -> List[dict]:
        pass

    def search_paginate(self, query: dict, limit: int) -> Iterator:
        pass

    def bulk(
        self,
        create_items: List[dict] = None,
        update_items: List[dict] = None,
        create_edges: List[dict] = None,
        delete_items: List[str] = None,
    ) -> None:
        pass

In [ ]:
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account

client = PodClient()
api = PodAPI(database_key=client.database_key, owner_key=client.owner_key)

In [ ]:
assert len(api.version)

In [ ]:
account = Account(identifier="abcd")
client.create(account)

True

In [ ]:
api.get_item(account.id)

[{'dateCreated': 1636472956588,
  'dateModified': 1636472956588,
  'dateServerModified': 1636472956588,
  'deleted': False,
  'id': '1726ba1773aaacdbd9e074ea7970009e',
  'identifier': 'abcd',
  'type': 'Account'}]

In [ ]:
# hide
from nbdev.export import *
notebook2script()